# 4. Decisoin Tree

* 문제
* 알고리즘 정의
* 구현
    * 공개된 데이터
    * 함수로 구현 -> 라이브러리 (비객체지향)
* 성능
    * 정확성, 빠르기
* 사례
    * house-votes-84.data 데이터 이용
    * 패키지 사용

## 4.1 문제

* 의사결정에 영향을 미치는 속성을 값에 따라 분기하여, 최종 결정에 이르도록 한다.
* 사다리타기와 같이 최종 결정이 명목 값을 가진다.
* 속성 값이 연속적이지 아닌 경우 사용한다.
* 몸무게와 같이 연속적인 값이 아닌 결정일 경우 사용한다.

## 4.2 알고리즘

### 4.2.1 ID3 algorithm

* input: Training Data, Attribute List
  * 학습데이터
  * 속성목록
* output: decision tree

Generate_decision_tree(Training Data, Attribute List)
* create a node N 시작 노드
* if samples are all of the same class C then
    return N as a leaf node labeled with class C; 모두 같은 클래스인 경우 분기하여 노드
* if attribute-list is empty then
    return N as a leaft node labeled with majority voting; 분기할 속성이 남아 있지 않으면 최빈으로 노드
* from among attributes in the attribute-list,
    select test attribute that leads to the highest information gain
    label node N with the test-attribute; IG가 가장 높은 속성을 선택하여 노드
* for each known value ai of test-attribute //partition the samples
    * grow a branch from node N for the condition test-attribute = ai 분기
    * let si be the set of samples in the samples for which test-attribute = ai 데이터분할
    * if si is empty then
        attach a leaf labelled with majority voting; 분할한 데이터가 공집합이면 최빈
    * else
        attach the node returned by Generate_decision_tree(si, Attribute List - test attribute); 재귀적으로 함수를 실행하고 그 결과 값을 받아서 노드
}

### 4.2.2 Shannon entorpy

* 정보이론 (Shannon, 1948)
* Entropy H(S)는 데이터에 포함된 불확실성, 엔트로피가 클수록 불확실성이 높음.
* log2를 취하면서 그 승수가 정보를 표현하는 bit수를 의미.
즉 4의 log2 값은 2 ($2^2$), 4개의 정보 값을 표현하려면 2 bits 필요.

* 참고
    * scipy.stats.entropy
    * Gini impurity

$$
\begin{align}
H(S) &= - \sum_{i=1}^{n}\ p_i\ log_2\ p_i\\
     &= - p_1 log_2\ p_1 - p_2 log_2\ p_2\ \ldots
\end{align}
$$

위 식에서 $p_i$는 사건i가 발생할 확률을 의미.
동전을 던지는 경우, 모두 앞면 또는 모두 뒷면은 entropy가 0 (즉 불확실성이 없는 purity)
반대로 반반씩 섞여 있는 경우 entropy는 가장 크다.
즉 엔트로피가 클수록 정보가 혼재되어 있다는 의미.
가장 유용한 정보는 엔트로피를 가장 많이 감소시키는 것.

```
H( (0.5,0.5) ) = -0.5 x log_2 0.5 - 0.5 x log_2 0.5 = 1
H( (0.5,0.5) ) = -0.7 x log_2 0.7 - 0.3 x log_2 0.3 = 0.88
H( (0.5,0.5) ) = -1.0 x log_2 1.0 - 0.0 x log_2 0.0 = 0
```

In [7]:
import math
print [-p*math.log(p,2) for p in [0.5,0.5]]
print [-p*math.log(p,2) for p in [0.7,0.3]]
try:
    [-p*math.log(p,2) for p in [1.0,0.0]]
except Exception as e:
    print e

[0.5, 0.5]
[0.3602012209808308, 0.5210896782498619]
math domain error


* 엔트로피 계산 - 문자열
    * 엔트로피는 부호화에 필요한 문자 당 최소 평균 이진값
        ```
        str = 'aabcddddefffg'
        ```

In [24]:
import numpy as np
import math
str = 'aabcddddefffg'
# 1) counts frequency
allChars=list(str)
charDict=dict()
for token in allChars:
    if charDict.has_key(token):
        charDict[token]+=1
    else:
        charDict[token]=1
# 2) computes entropy
entropy=0
allFreq=float(len(allChars))
for key in charDict.iterkeys():
    freq=charDict[key]
    prob=float(freq)/allFreq
    ent=-prob * math.log(prob,2)
    entropy=entropy+ent
    print "'{0}'\tFreq->{1}\tProb->{2}\teach entropy->{3} Entropy{4}".format(key,freq,prob,ent,entropy)


'a'	Freq->2	Prob->0.153846153846	each entropy->0.415452264329 Entropy0.415452264329
'c'	Freq->1	Prob->0.0769230769231	each entropy->0.284649209088 Entropy0.700101473417
'b'	Freq->1	Prob->0.0769230769231	each entropy->0.284649209088 Entropy0.984750682505
'e'	Freq->1	Prob->0.0769230769231	each entropy->0.284649209088 Entropy1.26939989159
'd'	Freq->4	Prob->0.307692307692	each entropy->0.523212220966 Entropy1.79261211256
'g'	Freq->1	Prob->0.0769230769231	each entropy->0.284649209088 Entropy2.07726132165
'f'	Freq->3	Prob->0.230769230769	each entropy->0.488187050174 Entropy2.56544837182


* 엔트로피 계산
    * 5건에서 yes 2건, no 3건

In [9]:
print "case:\tyes=2/5 no=3/5 (MiA p.42)"
print "\tcomputing entropy --> 0.97095059"
S=np.array((0.4, 0.6))
entropy=[ -p*math.log(p,2) for p in S ]
def computeEntropy(data):
    entropy=[-p*math.log(p,2) for p in data]
    return sum(entropy)
print "\tentropy={0}\n\tfunct={1}".format(entropy,computeEntropy(S))

case:	yes=2/5 no=3/5 (MiA p.42)
	computing entropy --> 0.97095059
	entropy=[0.52877123795494485, 0.44217935649972373]
	funct=0.970950594455


### 4.2.3 information gain

* 불확실성이 가장 낮은 속성으로 분기하기 위한 방법. 가장 높은 IG를 선택하여 분기함.
* 속성선정값 (attribute selection measure) 또는 분기적합도 (measure of the goodness of split)

$$
IG(A,S) = H(S) - \sum_{i=1}^m\ p(i)\ H(i)
$$

* 각 속성의 값으로, 그 발생확률을 계산 (위 식 $p(i)$)
* 그 속성 값의 엔트로피를 계산 (H(i)
* 위 1과 2를 가중하여 해당 속성의 엔트로피를 계산 (위 식의 우측 항)
* 기본엔트로피 (H(s))에서 공제하여 Information Gain 

## 4.3 구현

1. 데이터준비
2. create tree
    * 엔트로피 계산
    * Information Gain 계산
    * decision tree 구조 만듦
3. classify
4. 평가

### 4.3.1 데이터 준비

* 사례
    * source: Liu, Bing. Web data mining: exploring hyperlinks, contents, and usage data. Springer Science & Business Media, 2007.
    * 속성 4개 열, 속성 값은 문자열
    * 클래스 마지막 열
    * 테이블로 나타내면:

| age | has_job | own_house | credit_rating | class |
|-----|---------|-----------|---------------|-------|
| young | false | false | fair | no |

* numpy로 계산 (List로 처리하는 것과 비교)

In [1]:
import numpy as np

myV=np.array([['young', 'false', 'false', 'fair', 'No'],
           ['young', 'false', 'false', 'good', 'No'],
           ['young', 'true', 'false', 'good', 'Yes'],
           ['young', 'true', 'true', 'fair', 'Yes'],
           ['young', 'false', 'false', 'fair', 'No'],
           ['middle', 'false', 'false', 'fair', 'No'],
           ['middle', 'false', 'false', 'good', 'No'],
           ['middle', 'true', 'true', 'good', 'Yes'],
           ['middle', 'false', 'true', 'excellent', 'Yes'],
           ['middle', 'false', 'true', 'excellent', 'Yes'],
           ['old', 'false', 'true', 'excellent', 'Yes'],
           ['old', 'false', 'true', 'good', 'Yes'],
           ['old', 'true', 'false', 'good', 'Yes'],
           ['old', 'true', 'false', 'excellent', 'Yes'],
           ['old', 'false', 'false', 'fair', 'No']], 
          dtype='|S9')

print myV

[['young' 'false' 'false' 'fair' 'No']
 ['young' 'false' 'false' 'good' 'No']
 ['young' 'true' 'false' 'good' 'Yes']
 ['young' 'true' 'true' 'fair' 'Yes']
 ['young' 'false' 'false' 'fair' 'No']
 ['middle' 'false' 'false' 'fair' 'No']
 ['middle' 'false' 'false' 'good' 'No']
 ['middle' 'true' 'true' 'good' 'Yes']
 ['middle' 'false' 'true' 'excellent' 'Yes']
 ['middle' 'false' 'true' 'excellent' 'Yes']
 ['old' 'false' 'true' 'excellent' 'Yes']
 ['old' 'false' 'true' 'good' 'Yes']
 ['old' 'true' 'false' 'good' 'Yes']
 ['old' 'true' 'false' 'excellent' 'Yes']
 ['old' 'false' 'false' 'fair' 'No']]


### 4.3.2 엔트로피 계산

* 속성의 키를 찾아, 빈도를 계산
    * numpy bincount
        ```
        buildKeyCountVec(data)
        ```
    * 간편하게 collections을 이용할 수 있슴.
        ```
        from collections import Counter
        kc = Counter(data)
        ```
* 확률 계산
    ```
    computeProbVec(kc)
    ```

* 엔트로피 계산
    ```
    entropyVec(data)
    ```

In [34]:
#학습데이터 첫째 속성(나이)의 엔트로피계산을 위한 확률 구함.
#키를 찾아, 빈도를 계산
a=myV[:,0]
keys=np.unique(a)
bins=keys.searchsorted(myV[:,0])
freq=np.bincount(bins)
print "나이{0} 빈도{1}".format(keys,freq)

나이['middle' 'old' 'young'] 빈도[5 5 5]


In [42]:
def buildKeyCountVec(data):
    import numpy as np
    keys=np.unique(data)
    bins=keys.searchsorted(data)
    return np.vstack([keys,np.bincount(bins)])

myX=[2,2,8,7,5,3,1,1,2,2,8,7,5,3,9,7]
buildKeyCountVec(myX)
print "첫째 속성의 키/빈도수\n{0}".format(buildKeyCountVec(myV[:,0]))

첫째 속성의 키/빈도수
[['middle' 'old' 'young']
 ['5' '5' '5']]


In [52]:
#전체 데이터에 대해 엔트로피를 계산하면
#1) 확률 계산
#Yes: 9/15 No: 6/15
print "마지막 컬럼 클래스의 키/빈도수\n{0}".format(findKeyCounts(myV[:,-1]))

#전체빈도수 계산.
kcV=buildKeyCountVec(myV[:,-1])
#array는 문자열이 섞이면 수도 문자열로 casting.
allFreq=kcV[1,:].astype('int').sum()
#확률계산 - vector연산이므로 for-loop가 필요없슴.
prob=kcV[1,:].astype('float')/allFreq
print "확률 {0} 전체빈도 {1}".format(prob,allFreq)

#2) 엔트로피를 계산
print "엔트로피는 {0}".format(sum([-p*math.log(p,2) for p in prob]))

마지막 컬럼 클래스의 키/빈도수
[['No' 'Yes']
 ['6' '9']]
확률 [ 0.4  0.6] 전체빈도 15
엔트로피는 0.970950594455


In [49]:
def computeProbVec(kc):
    allFreq=kc[1,:].astype('int').sum()
    prob=kc[1,:].astype('float')/allFreq
    print "[from computeProbVec] prob {0} all freq {1}".format(prob,allFreq)
    return prob

def entropyVec(data):
    import math
    kc=buildKeyCountVec(data)
    prob=computeProbVec(kc)
    entropy=computeEntropy(prob)
    return entropy

print "함수를 이용한 엔트로피는 {0}".format(entropyVec(myV[:,-1]))

[from computeProbVec] prob [ 0.4  0.6] all freq 15
함수를 이용한 엔트로피는 0.970950594455


### 4.3.3 Information Gain 계산

* age 속성으로 info gain을 손으로 계산해 보기

In [57]:
# select best feature
# 1) age --> 0.888
#H(age)=5/15 * H(age=young) + 5/15 * H(age=middle) + 5/15 * H(age=old)
# 1-1) select 'young'
#array([['young', 'false', 'false', 'fair', 'No'],
#       ['young', 'false', 'false', 'good', 'No'],
#       ['young', 'true', 'false', 'good', 'Yes'],
#       ['young', 'true', 'true', 'fair', 'Yes'],
#       ['young', 'false', 'false', 'fair', 'No']])
subV=myV[myV[:,0]=='young']
print buildKeyCountVec(subV[:,-1])
#array([['No', 'Yes'],['3', '2']])
freqYoung=len(subV)
entYoung=sum([-p*math.log(p,2) for p in (3/5.,2/5.)])
#0.9709505944546686
# 1-2) select 'middle'
#array([['middle', 'false', 'false', 'fair', 'No'],
#       ['middle', 'false', 'false', 'good', 'No'],
#       ['middle', 'true', 'true', 'good', 'Yes'],
#       ['middle', 'false', 'true', 'excellent', 'Yes'],
#       ['middle', 'false', 'true', 'excellent', 'Yes']]) 
subV=myV[myV[:,0]=='middle']
print buildKeyCountVec(subV[:,-1])
#array([['No', 'Yes'],['2', '3']]) 
freqMiddle=len(subV)
entMiddle=sum([-p*math.log(p,2) for p in (2/5.,3/5.)])
#0.9709505944546686
# 1-3) select 'old'
subV=myV[myV[:,0]=='old']
print buildKeyCountVec(subV[:,-1])
#array([['No', 'Yes'],['1', '4']])
freqOld=len(subV)
entOld=sum([-p*math.log(p,2) for p in (1/5.,4/5.)])
#0.7219280948873623
# 2) compute prob and entropy
probYoung=float(freqYoung)/len(myV)
entPYoung=probYoung*entYoung
print "Young Entropy",entPYoung,"=",probYoung," x ",entYoung
entAge=entPYoung

probMiddle=float(freqMiddle)/len(myV)
entPMiddle=probMiddle*entMiddle
print "Middle Entropy",entPMiddle,"=",probMiddle," x ",entMiddle
entAge+=entPMiddle

probOld=float(freqOld)/len(myV)
entPOld=probOld*entOld
print "Old Entropy",entPOld,"=",probOld," x ",entOld
entAge+=entPOld

print "Age Entropy:",entAge,"=",entPYoung,"+",entPMiddle,"+",entPOld
baseEntropy=sum([-p*math.log(p,2) for p in (9/15.,6/15.)])
# 3) IG
print "Class Entropy:",baseEntropy
print "age info gain:",baseEntropy-entAge,"=",baseEntropy,"-",entAge
# 4) pick the best
#IG(D,age) = 0.971 - 0.888 = 0.083
#IG(D,own_house) = 0.971 - 0.551 = 0.420
#IG(D,has_job) = 0.971 - 0.647 = 0.324
#IG(D,credit_rating) = 0.971 - 0.608 = 0.363
#pick own_house as the largest IG


[['No' 'Yes']
 ['3' '2']]
[['No' 'Yes']
 ['2' '3']]
[['No' 'Yes']
 ['1' '4']]
Young Entropy 0.323650198152 = 0.333333333333  x  0.970950594455
Middle Entropy 0.323650198152 = 0.333333333333  x  0.970950594455
Old Entropy 0.240642698296 = 0.333333333333  x  0.721928094887
Age Entropy: 0.887943094599 = 0.323650198152 + 0.323650198152 + 0.240642698296
Class Entropy: 0.970950594455
age info gain: 0.0830074998558 = 0.970950594455 - 0.887943094599


gain(D, Age) = 0.971 - 0.888 = 0.083
gain(D, Has_job) = 0.971 - 0.647 = 0.324
gain(D, Own_house) = 0.971 - 0.551 = 0.420
gain(D, Credit_rating) = 0.971 - 0.608 = 0.363

* information gain 함수
    * feature별로 계산한다.
    * best feature를 고른다.


In [61]:
baseEntropy=entropyVec(myV[:,-1]) # class labels
nFeatures=myV.shape[1]-1 # n of features except the class label
igV=np.zeros([nFeatures]) # empty array of features
aFeature=0 # age -> the first feature
kcV=buildKeyCountVec(myV[:,aFeature])
nKeys=kcV.shape[1] # 3 keys -> old,middle,young
allFreq=kcV[1,:].astype('int').sum() #n=15
prob=kcV[1,:].astype('float')/allFreq # [5/15,5/15,5/15]
print "\tage p={0} allFreq={1}".format(prob,allFreq)
for aKey in range(nKeys):
    keyToSearch=kcV[0][aKey]
    subData=myV[myV[:,aFeature]==keyToSearch]
    igV[aFeature]+=prob[aKey]*entropyVec(subData[:,-1])
    print "keyToSearch={0} Entropy={1}".format(keyToSearch,igV)
igV[aFeature]=baseEntropy-igV[aFeature]
print "\tage info gain={0}".format(igV)


[from computeProbVec] prob [ 0.4  0.6] all freq 15
	age p=[ 0.33333333  0.33333333  0.33333333] allFreq=15
[from computeProbVec] prob [ 0.4  0.6] all freq 5
keyToSearch=middle Entropy=[ 0.3236502  0.         0.         0.       ]
[from computeProbVec] prob [ 0.2  0.8] all freq 5
keyToSearch=old Entropy=[ 0.5642929  0.         0.         0.       ]
[from computeProbVec] prob [ 0.6  0.4] all freq 5
keyToSearch=young Entropy=[ 0.88794309  0.          0.          0.        ]
	age info gain=[ 0.0830075  0.         0.         0.       ]


In [68]:
def getInfoGainVec(data):
    import numpy as np
    nFeature=data.shape[1]-1 # except the last column
    InfoGain=np.zeros([nFeature]) # by feature
    for item in range(nFeature):
        InfoGain[item]=computeInfoGain(data[:,[item,-1]])
        print "[getInfoGainVec] {0}th InfoGain={1}".format(item,InfoGain)
    return InfoGain

def computeInfoGain(data):
    di=0 # the first column = data column
    classEntropy=entropyVec(data[:,-1]) # class labels
    kc=buildKeyCountVec(data[:,di])
    nKey=kc.shape[1] #n of unique keys
    prob=computeProbVec(kc)
    rawInfoGain=0.
    for item in range(nKey):
        keyToSearch=kc[0][item]
        subData=split2DVec(data,di,keyToSearch)
        classEntropyByItem=entropyVec(subData[:,-1])
        rawInfoGain+=prob[item]*classEntropyByItem
    InfoGain=classEntropy-rawInfoGain
    return InfoGain

def split2DVec(data,col,query):
    subData=data[data[:,col]==query]
    return subData

print getInfoGainVec(myV)

[from computeProbVec] prob [ 0.4  0.6] all freq 15
[from computeProbVec] prob [ 0.33333333  0.33333333  0.33333333] all freq 15
[from computeProbVec] prob [ 0.4  0.6] all freq 5
[from computeProbVec] prob [ 0.2  0.8] all freq 5
[from computeProbVec] prob [ 0.6  0.4] all freq 5
[getInfoGainVec] 0th InfoGain=[ 0.0830075  0.         0.         0.       ]
[from computeProbVec] prob [ 0.4  0.6] all freq 15
[from computeProbVec] prob [ 0.66666667  0.33333333] all freq 15
[from computeProbVec] prob [ 0.6  0.4] all freq 10
[from computeProbVec] prob [ 1.] all freq 5
[getInfoGainVec] 1th InfoGain=[ 0.0830075  0.3236502  0.         0.       ]
[from computeProbVec] prob [ 0.4  0.6] all freq 15
[from computeProbVec] prob [ 0.6  0.4] all freq 15
[from computeProbVec] prob [ 0.66666667  0.33333333] all freq 9
[from computeProbVec] prob [ 1.] all freq 6
[getInfoGainVec] 2th InfoGain=[ 0.0830075   0.3236502   0.41997309  0.        ]
[from computeProbVec] prob [ 0.4  0.6] all freq 15
[from computeProbV

### 4.3.4 Decision Tree



In [4]:
# %load -r 747-752 src/pystat/sfun.py
def swapColRow(data):
    dataCol=list()
    for c in range(len(data[0])):
        col=[eachRow[c] for eachRow in data]
        dataCol.append(col)
    return dataCol

def splitListByRow(data,row,query):
    subData=list()
    index=list()
    for i,v in enumerate(data[row]):
        if v==query:
            index.append(i)
    for j in range(len(data)):
        subDataByRow=list()
        for i in index:
            subDataByRow.append(data[j][i])
        subData.append(subDataByRow)
    return subData


def createTree(keyList,n=0):
    from collections import defaultdict
    nStop=len(keyList)-1
    if n==nStop:
        cLables=keyList[-1]
        return cLables
    # d grows dynamically
    d=defaultdict()
    parentKeyList=set(keyList[n])
    for key in parentKeyList:
        subKeyList=splitListByRow(keyList,n,key)
        d.update({key:createTree(subKeyList,n+1)})
    return d

myL=myV.tolist()
myLCol=swapColRow(myL)
myTree=createTree(myLCol)
print myLCol

[['young', 'young', 'young', 'young', 'young', 'middle', 'middle', 'middle', 'middle', 'middle', 'old', 'old', 'old', 'old', 'old'], ['false', 'false', 'true', 'true', 'false', 'false', 'false', 'true', 'false', 'false', 'false', 'false', 'true', 'true', 'false'], ['false', 'false', 'false', 'true', 'false', 'false', 'false', 'true', 'true', 'true', 'true', 'true', 'false', 'false', 'false'], ['fair', 'good', 'good', 'fair', 'fair', 'fair', 'good', 'good', 'excellent', 'excellent', 'excellent', 'good', 'good', 'excellent', 'fair'], ['No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']]


In [5]:
# %load -r 755-764 src/pystat/sfun.py
def dPrint(d,depth=0):
    depth+=1
    for k, v in d.iteritems():
        print "+"*depth,
        if isinstance(v, dict):
            print "{0} {1}".format(depth,k)
            dPrint(v,depth)
        else:
            print "{0} {1}: {2}".format(depth, k, v)

dPrint(myTree)

+ 1 middle
++ 2 false
+++ 3 false
++++ 4 good: ['No']
++++ 4 fair: ['No']
+++ 3 true
++++ 4 excellent: ['Yes', 'Yes']
++ 2 true
+++ 3 true
++++ 4 good: ['Yes']
+ 1 old
++ 2 false
+++ 3 true
++++ 4 good: ['Yes']
++++ 4 excellent: ['Yes']
+++ 3 false
++++ 4 fair: ['No']
++ 2 true
+++ 3 false
++++ 4 good: ['Yes']
++++ 4 excellent: ['Yes']
+ 1 young
++ 2 false
+++ 3 false
++++ 4 good: ['No']
++++ 4 fair: ['No', 'No']
++ 2 true
+++ 3 false
++++ 4 good: ['Yes']
+++ 3 true
++++ 4 fair: ['Yes']


### 4.3.5 분류



In [11]:
print myTree['middle']['false']['false']['good']

['No']

## 4.4 사례

### 4.4.1 scikit ML 패키지를 사용하여:

* see
http://www.netinstructions.com/machine-learning-with-decision-trees/

In [72]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
clf.predict([[2., 2.]])

array([1])

### 4.4.2 교재

* MiA

In [71]:
import os

myhome=os.path.expanduser('~')
ch3wd=os.path.join(myhome,'Code/git/else/machinelearninginaction/Ch03')
os.chdir(ch3wd)
import trees
myDat,labels=trees.createDataSet()
#myDat [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
#labels ['no surfacing', 'flippers']
print "entropy=",trees.calcShannonEnt(myDat) #0.9709505944546686

#inside calcShannonEnt: 학습데이터 1개 행의 클래스의 빈도 계산
numEntries=len(myDat)
labelCounts={}
currentLabel=myDat[0][-1] #[1, 1, 'yes']의 마지막 'yes'
if currentLabel not in labelCounts.keys():
    labelCounts[currentLabel]=0
labelCounts[currentLabel]+=1
print labelCounts #{'yes': 1}

print trees.chooseBestFeatureToSplit(myDat)

# create tree as a dictionary and search
#createTree p.48
myTree=trees.createTree(myDat,labels)
#{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
print labels #createTree하면 labels lost ['flippers'], so reload!!!

#insdie classify p.56
myDat,labels=trees.createDataSet() #now labels ['no surfacing', 'flippers']
firstStr=myTree.keys()[0] #'no surfacing'
secondDict=myTree[firstStr] #{0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}
featIndex=labels.index(firstStr) #0
secondDict.keys() #[0, 1]
secondDict #{0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}
testVec=[1,0]
key=secondDict.keys()[0] #0
print testVec[featIndex]==key #False
print "testVec={0}을 분류하면 결과 class label={1}".format(testVec,secondDict[0])

# sample data로
myV=np.array([['young', 'false', 'false', 'fair', 'No'],
       ['young', 'false', 'false', 'good', 'No'],
       ['young', 'true', 'false', 'good', 'Yes'],
       ['young', 'true', 'true', 'fair', 'Yes'],
       ['young', 'false', 'false', 'fair', 'No'],
       ['middle', 'false', 'false', 'fair', 'No'],
       ['middle', 'false', 'false', 'good', 'No'],
       ['middle', 'true', 'true', 'good', 'Yes'],
       ['middle', 'false', 'true', 'excellent', 'Yes'],
       ['middle', 'false', 'true', 'excellent', 'Yes'],
       ['old', 'false', 'true', 'excellent', 'Yes'],
       ['old', 'false', 'true', 'good', 'Yes'],
       ['old', 'true', 'false', 'good', 'Yes'],
       ['old', 'true', 'false', 'excellent', 'Yes'],
       ['old', 'false', 'false', 'fair', 'No']], 
      dtype='|S9')
dat=myV[:,:-1].tolist()
lab=myV[:,-1].tolist()
print trees.createTree(dat,lab)

entropy= 0.970950594455
{'yes': 1}
0
['flippers']
False
testVec=[1, 0]을 분류하면 결과 class label=no
{'No': {'middle': {'Yes': {'false': {'No': {'good': 'good', 'fair': 'fair'}}, 'true': {'No': {'true': 'good', 'false': 'excellent'}}}}, 'old': {'No': {'false': {'Yes': {'true': 'good', 'false': 'fair'}}, 'true': {'No': {'good': 'good', 'excellent': 'excellent'}}}}, 'young': {'Yes': {'false': {'No': {'false': 'fair', 'true': 'good'}}, 'true': 'fair'}}}}
